In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
datasets = ['BC15','Huongthom', 'Nep87','Q5','Thienuu','Xi23']
features = ['basic','gist','glcm','lbp']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=69)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [49]:
models = {
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'SVM': SVC(kernel='rbf', random_state=69),
    'Random Forest': RandomForestClassifier(n_estimators=40, random_state=69)
}

results = []

In [ ]:
for dataset in datasets:
    for feature_set in features:
        dataset_path = f"/home/duyle/Rice_photos/features_extracted/{feature_set.lower()}/{feature_set}_{dataset}.csv"
        
        df = pd.read_csv(dataset_path)
        
        X = df.drop(['Label'], axis=1)
        y = df['Label']
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=69, stratify=y)

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
            
        for name, model in models.items():
            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)
            
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred, average='weighted')
            recall = recall_score(y_test, y_pred, average='weighted')
            f1 = f1_score(y_test, y_pred, average='weighted')
            cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5)
            
            results.append({
                "Model": name,
                "Dataset": dataset,
                "Features": feature_set,
                "Accuracy": accuracy,
                "Precision": precision,
                "Recall": recall,
                "F1 Score": f1,
            })

In [44]:
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5)
    
    results.append({
        "Model": name,
        "Current dataset": "BC-15",
        "Features": "Basic",
        "Test Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1,
    })

KeyboardInterrupt: 

In [68]:
df

,Model,Dataset,Features,Accuracy,Precision,Recall,F1 Score
0,KNN,BC15,basic,0.8402,0.8466,0.8402,0.8395
1,SVM,BC15,basic,0.8987,0.9039,0.8987,0.8984
2,Random Forest,BC15,basic,0.8526,0.8543,0.8526,0.8524
3,KNN,BC15,gist,0.6367,0.6368,0.6367,0.6367
4,SVM,BC15,gist,0.6960,0.6963,0.6960,0.6959
...,...,...,...,...,...,...,...
67,SVM,Xi23,glcm,0.7727,0.7786,0.7727,0.7693
68,Random Forest,Xi23,glcm,0.7580,0.7579,0.7580,0.7580
69,KNN,Xi23,lbp,0.6681,0.6676,0.6681,0.6653
70,SVM,Xi23,lbp,0.6886,0.6896,0.6886,0.6848


In [53]:
df = pd.DataFrame(results)


In [57]:
df = df.round(4)

In [72]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)


In [74]:
df

,Model,Dataset,Features,Accuracy,Precision,Recall,F1 Score
0,KNN,BC15,basic,0.8402,0.8466,0.8402,0.8395
1,SVM,BC15,basic,0.8987,0.9039,0.8987,0.8984
2,Random Forest,BC15,basic,0.8526,0.8543,0.8526,0.8524
3,KNN,BC15,gist,0.6367,0.6368,0.6367,0.6367
4,SVM,BC15,gist,0.6960,0.6963,0.6960,0.6959
5,Random Forest,BC15,gist,0.7702,0.7707,0.7702,0.7701
6,KNN,BC15,glcm,0.6738,0.6766,0.6738,0.6726
7,SVM,BC15,glcm,0.7512,0.7512,0.7512,0.7512
8,Random Forest,BC15,glcm,0.7166,0.7167,0.7166,0.7166
9,KNN,BC15,lbp,0.5568,0.5570,0.5568,0.5567


In [76]:
df[df[Features == 'basic']]

NameError: name 'Features' is not defined

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
    print(df)

ValueError: Value must be a nonnegative integer or None

In [59]:
df.to_csv('results.csv',index=False)